In [1]:
import numpy as np
import pandas as pd
import requests 
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [14]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

In [15]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [16]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_uri = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_uri).json()
    for symbol in symbol_string.split(','):
        final_dataframe = pd.concat([final_dataframe, pd.DataFrame(
        {
            final_dataframe.columns[0]: [symbol],
            final_dataframe.columns[1]: [data[symbol]['quote']['latestPrice']],
            final_dataframe.columns[2]: [data[symbol]['quote']['marketCap']],
            final_dataframe.columns[3]: ["N/A"],
        }
        )], ignore_index=True)
final_dataframe
        

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS
GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INF

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,129.88,38449749232,N/A
1,AAL,13.71,8910141161,N/A
2,AAP,171.34,10300659070,N/A
3,AAPL,147.27,2366739647040,N/A
4,ABBV,147.06,260016270555,N/A
...,...,...,...,...
496,YUM,111.03,31592750888,N/A
497,ZBH,107.39,22532521260,N/A
498,ZBRA,263.04,13622826081,N/A
499,ZION,47.82,7195541153,N/A


In [58]:
portfolio_size = input('Enter the value of your portfolio:')
try: 
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter the value of your portfolio:')

100000000.0


In [59]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe["Stock Price"][i])

final_dataframe


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,129.88,38449749232,1536
1,AAL,13.71,8910141161,14558
2,AAP,171.34,10300659070,1164
3,AAPL,147.27,2366739647040,1355
4,ABBV,147.06,260016270555,1357
...,...,...,...,...
496,YUM,111.03,31592750888,1797
497,ZBH,107.39,22532521260,1858
498,ZBRA,263.04,13622826081,758
499,ZION,47.82,7195541153,4174


In [87]:
writer = pd.ExcelWriter('Stocks.xlsx', engine='xlsxwriter', options={'strings_to_urls': False})
final_dataframe.to_excel(writer, 'Stocks', index=False)

/var/folders/y3/jmcnq4pj2fqc4yw48rkvt1zr0000gn/T/ipykernel_8835/2854146335.py:1: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  writer = pd.ExcelWriter('Stocks.xlsx', engine='xlsxwriter', options={'strings_to_urls': False})


In [89]:
bg_colour = '#C0C999'
font_colour = '#000000'


string_format = writer.book.add_format(
    {
        'font_color': font_colour,
        'bg_color': bg_colour,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_colour,
        'bg_color': bg_colour,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_colour,
        'bg_color': bg_colour,
        'border': 1
    }
)


In [90]:
writer.sheets['Stocks'].set_column('A:A', 18, string_format)
writer.sheets['Stocks'].set_column('B:B', 18, dollar_format)
writer.sheets['Stocks'].set_column('C:C', 18, dollar_format)
writer.sheets['Stocks'].set_column('D:D', 18, integer_format)


0

In [91]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}
for column in column_formats.keys():
    writer.sheets['Stocks'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Stocks'].write(f'{column}1', column_formats[column][0])


In [92]:
writer.save()